# SD - SR 

The following notebook allows to replicate the simulation as describe in Alamia et al. (2019). CNN and a Siamese networks are compared when performing identity (SD) and Spatial Relationship (SR) tasks. See Alamia et al. (2019) for further details.

After having imported a bunch of packages, we load the data from a previously genereated Matlab file.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

import time
import csv
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D, Conv2D, ZeroPadding2D, Activation, Input, concatenate, Add, Multiply

from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras import regularizers, optimizers
from keras.utils import np_utils
np.random.seed(1234)

import tensorflow as tf 
import keras
from keras.layers.merge import _Merge

import scipy.io as sio
from scipy import stats


In [3]:
def getImagesNlabelsSiam() :

    infile = sio.loadmat('siamese1000val.mat')
    
    allTrialsTr = infile.get('allTrialsTr')
    SRconditionsTr=infile.get('SRconditionsTr')
    SDconditionsTr=infile.get('SDconditionsTr')
    
    allTrialsVal = infile.get('allTrialsVal')
    SRconditionsVal=infile.get('SRconditionsVal')
    SDconditionsVal=infile.get('SDconditionsVal')
    
    allTrialsTe = infile.get('allTrialsTe')
    SRconditionsTe=infile.get('SRconditionsTe')
    SDconditionsTe=infile.get('SDconditionsTe')

    #shuffling training
    perm = np.arange(allTrialsTr.shape[3])  
    np.random.shuffle(perm)  
    allTrialsTrPerm = allTrialsTr[:,:,:,perm] 
    SRconditionsTrPerm = SRconditionsTr[:,perm] 
    SDconditionsTrPerm = SDconditionsTr[:,perm] 
   
    #providing it with the right dimensions  training
    allTrialsTrRearranged=np.moveaxis(allTrialsTrPerm,-1,0)
    SDconditionsTrRearranged=np.moveaxis(SDconditionsTrPerm,-1,0)
    SRconditionsTrRearranged=np.moveaxis(SRconditionsTrPerm,-1,0)
    allTrialsTrRearranged2 = np.expand_dims(allTrialsTrRearranged,axis=-1)
    allTrialsTrRearranged2 = allTrialsTrRearranged
    
    
    #shuffling validation
    perm = np.arange(allTrialsVal.shape[3])  
    np.random.shuffle(perm)  
    allTrialsValPerm = allTrialsVal[:,:,:,perm] 
    SRconditionsValPerm = SRconditionsVal[:,perm] 
    SDconditionsValPerm = SDconditionsVal[:,perm] 
   
    #providing it with the right dimensions validation
    allTrialsValRearranged=np.moveaxis(allTrialsValPerm,-1,0)
    SDconditionsValRearranged=np.moveaxis(SDconditionsValPerm,-1,0)
    SRconditionsValRearranged=np.moveaxis(SRconditionsValPerm,-1,0)
    allTrialsValRearranged2 = np.expand_dims(allTrialsValRearranged,axis=-1)
    allTrialsValRearranged2 = allTrialsValRearranged
    
    
    #shuffling testing
    perm = np.arange(allTrialsTe.shape[3])  
    np.random.shuffle(perm)  
    allTrialsTePerm = allTrialsTe[:,:,:,perm] 
    SRconditionsTePerm = SRconditionsTe[:,perm] 
    SDconditionsTePerm = SDconditionsTe[:,perm] 
   
    #providing it with the right dimensions  testing
    allTrialsTeRearranged=np.moveaxis(allTrialsTePerm,-1,0)
    SDconditionsTeRearranged=np.moveaxis(SDconditionsTePerm,-1,0)
    SRconditionsTeRearranged=np.moveaxis(SRconditionsTePerm,-1,0)
    allTrialsTeRearranged2 = np.expand_dims(allTrialsTeRearranged,axis=-1)
    allTrialsTeRearranged2 = allTrialsTeRearranged
    
    return {'imagesTr':allTrialsTrRearranged2,'SRlabelsTr':SRconditionsTrRearranged,'SDlabelsTr':SDconditionsTrRearranged,'imagesVal':allTrialsValRearranged2,'SRlabelsVal':SRconditionsValRearranged,'SDlabelsVal':SDconditionsValRearranged,'imagesTe':allTrialsTeRearranged2,'SRlabelsTe':SRconditionsTeRearranged,'SDlabelsTe':SDconditionsTeRearranged}

In [4]:
def getImagesNlabels() :

    #infile = sio.loadmat('artVisDataset.mat')
    infile = sio.loadmat('cnn1000val.mat')

    allTrialsTr = infile.get('allTrialsTr')
    SRconditionsTr=infile.get('SRconditionsTr')
    SDconditionsTr=infile.get('SDconditionsTr')
    
    allTrialsVal = infile.get('allTrialsVal')
    SRconditionsVal=infile.get('SRconditionsVal')
    SDconditionsVal=infile.get('SDconditionsVal')
    
    allTrialsTe = infile.get('allTrialsTe')
    SRconditionsTe=infile.get('SRconditionsTe')
    SDconditionsTe=infile.get('SDconditionsTe')
    

    #shuffling training
    perm = np.arange(allTrialsTr.shape[2])  
    np.random.shuffle(perm)  
    allTrialsTrPerm = allTrialsTr[:,:,perm] 
    SRconditionsTrPerm = SRconditionsTr[:,perm] 
    SDconditionsTrPerm = SDconditionsTr[:,perm] 
   
    #providing it with the right dimensions  training
    allTrialsTrRearranged=np.moveaxis(allTrialsTrPerm,-1,0)
    SDconditionsTrRearranged=np.moveaxis(SDconditionsTrPerm,-1,0)
    SRconditionsTrRearranged=np.moveaxis(SRconditionsTrPerm,-1,0)
    allTrialsTrRearranged2 = np.expand_dims(allTrialsTrRearranged,axis=-1)
    
        
    #shuffling validation
    perm = np.arange(allTrialsVal.shape[2])  
    np.random.shuffle(perm)  
    allTrialsValPerm = allTrialsVal[:,:,perm] 
    SRconditionsValPerm = SRconditionsVal[:,perm] 
    SDconditionsValPerm = SDconditionsVal[:,perm] 
   
    #providing it with the right dimensions validation
    allTrialsValRearranged=np.moveaxis(allTrialsValPerm,-1,0)
    SDconditionsValRearranged=np.moveaxis(SDconditionsValPerm,-1,0)
    SRconditionsValRearranged=np.moveaxis(SRconditionsValPerm,-1,0)
    allTrialsValRearranged2 = np.expand_dims(allTrialsValRearranged,axis=-1)
    allTrialsValRearranged2 = allTrialsValRearranged
    
    
    #shuffling testing
    perm = np.arange(allTrialsTe.shape[2])  
    np.random.shuffle(perm)  
    allTrialsTePerm = allTrialsTe[:,:,perm] 
    SRconditionsTePerm = SRconditionsTe[:,perm] 
    SDconditionsTePerm = SDconditionsTe[:,perm] 
   
    #providing it with the right dimensions  testing
    allTrialsTeRearranged=np.moveaxis(allTrialsTePerm,-1,0)
    SDconditionsTeRearranged=np.moveaxis(SDconditionsTePerm,-1,0)
    SRconditionsTeRearranged=np.moveaxis(SRconditionsTePerm,-1,0)
    allTrialsTeRearranged2 = np.expand_dims(allTrialsTeRearranged,axis=-1)
    
    return {'imagesTr':allTrialsTrRearranged2,'SRlabelsTr':SRconditionsTrRearranged,'SDlabelsTr':SDconditionsTrRearranged,'imagesTe':allTrialsTeRearranged2,'SRlabelsTe':SRconditionsTeRearranged,'SDlabelsTe':SDconditionsTeRearranged}

In the next two cells we define the Siamese Network and the convolutional one. Note that in the Siamese network we redefine the class Substract.

In [5]:
class Subtract(_Merge):

    def build(self, input_shape):
        super(Subtract, self).build(input_shape)
        if len(input_shape) != 2:
            raise ValueError('A `Subtract` layer should be called '
                             'on exactly 2 inputs')

    def _merge_function(self, inputs):
        if len(inputs) != 2:
            raise ValueError('A `Subtract` layer should be called '
                             'on exactly 2 inputs')
        return inputs[0] - inputs[1]


def get_siamese_model(images):

    # Define the tensors for the two input images
#     left_input = Input(images[:,:,:,0].shape)
#     right_input = Input(images[:,:,:,1].shape)
#     left_input = np.expand_dims(left_input,axis=-1)
#     right_input = np.expand_dims(right_input,axis=-1)
#     left_input = Input(left_input.shape)
#     right_input = Input(right_input.shape)
    
    input_shape = (images.shape[1], images.shape[2], 1)
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    
    # Convolutional Neural Network
    model = Sequential()

    model.add(Conv2D(4, (2, 2), strides=1, input_shape=(images.shape[1], images.shape[2], 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))   
    model.add(Dropout(0.3))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    combineFeatures2 = Subtract()([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction1 = Dense(128,activation='relu')(combineFeatures2)
    prediction1do = Dropout(.3)(prediction1)
    prediction2 = Dense(128,activation='relu')(prediction1)
    prediction2do = Dropout(.3)(prediction2)
    prediction = Dense(1,activation='sigmoid')(prediction2do)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(lr=0.0001))
    
    # return the model
    return siamese_net

In [6]:
def buildingTheNetwork(images) :
    
    model = Sequential()

    model.add(Conv2D(4, (2, 2), strides=1, input_shape=(images.shape[1], images.shape[2], 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))   
    model.add(Dropout(0.2))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(lr=0.0001),)

    return model

Here we finally run the code

In [2]:
K.clear_session()

data=getImagesNlabels() 
imagesTr=data['imagesTr']
SRlabelsTr=data['SRlabelsTr']
SDlabelsTr=data['SDlabelsTr']

imagesTe=data['imagesTe']
SRlabelsTe=data['SRlabelsTe']
SDlabelsTe=data['SDlabelsTe']


dataSiam=getImagesNlabelsSiam() 
imagesTrSiam=dataSiam['imagesTr']
SRlabelsTrSiam=dataSiam['SRlabelsTr']
SDlabelsTrSiam=dataSiam['SDlabelsTr']

imagesValSiam=dataSiam['imagesVal']
SRlabelsValSiam=dataSiam['SRlabelsVal']
SDlabelsValSiam=dataSiam['SDlabelsVal']

imagesTeSiam=dataSiam['imagesTe']
SRlabelsTeSiam=dataSiam['SRlabelsTe']
SDlabelsTeSiam=dataSiam['SDlabelsTe']


imagesTrNewL=imagesTrSiam[:,:,:,0]
imagesTrNewR=imagesTrSiam[:,:,:,1]
imagesTrNewL = np.expand_dims(imagesTrNewL,axis=-1)
imagesTrNewR = np.expand_dims(imagesTrNewR,axis=-1)

imagesValNewL=imagesValSiam[:,:,:,0]
imagesValNewR=imagesValSiam[:,:,:,1]
imagesValNewL = np.expand_dims(imagesValNewL,axis=-1)
imagesValNewR = np.expand_dims(imagesValNewR,axis=-1)

imagesTeNewL=imagesTeSiam[:,:,:,0]
imagesTeNewR=imagesTeSiam[:,:,:,1]
imagesTeNewL = np.expand_dims(imagesTeNewL,axis=-1)
imagesTeNewR = np.expand_dims(imagesTeNewR,axis=-1)

networkSR=buildingTheNetwork(imagesTr) 
networkSD=buildingTheNetwork(imagesTr) 

siamNetSD=get_siamese_model(imagesTrSiam) 

networkSD.summary()
networkSR.summary()
siamNetSD.summary()

NameError: name 'K' is not defined

and finally we fit the models!

In [13]:
historySR = networkSR.fit(
            imagesTr, SRlabelsTr,
            batch_size=50, epochs=70, shuffle=False) 

historySD = networkSD.fit(
            imagesTr, SDlabelsTr,
            batch_size=50, epochs=70, shuffle=False) 

historySiam = siamNetSD.fit(
            [imagesTrNewR, imagesTrNewL], [SDlabelsTrSiam],
            batch_size=50, epochs=70, validation_split=0.0, validation_data=([imagesValNewR, imagesValNewL], [SDlabelsValSiam]), shuffle=False) 


Epoch 1/70
1000/1000 [==============================] - 4s - loss: 0.6619 - acc: 0.7280     
Epoch 2/70
1000/1000 [==============================] - 3s - loss: 0.5487 - acc: 0.9170     
Epoch 3/70
1000/1000 [==============================] - 3s - loss: 0.3653 - acc: 0.9310     
Epoch 4/70
1000/1000 [==============================] - 4s - loss: 0.2223 - acc: 0.9410     
Epoch 5/70
1000/1000 [==============================] - 4s - loss: 0.1490 - acc: 0.9510     
Epoch 6/70
1000/1000 [==============================] - 3s - loss: 0.1163 - acc: 0.9550     
Epoch 7/70
1000/1000 [==============================] - 3s - loss: 0.0770 - acc: 0.9780     
Epoch 8/70
1000/1000 [==============================] - 4s - loss: 0.0590 - acc: 0.9870     
Epoch 9/70
1000/1000 [==============================] - 4s - loss: 0.0575 - acc: 0.9830     
Epoch 10/70
1000/1000 [==============================] - 4s - loss: 0.0362 - acc: 0.9940     
Epoch 11/70
1000/1000 [==============================] - 4s - loss: 0

1000/1000 [==============================] - 4s - loss: 0.4792 - acc: 0.7750     
Epoch 16/70
1000/1000 [==============================] - 4s - loss: 0.4445 - acc: 0.7980     
Epoch 17/70
1000/1000 [==============================] - 4s - loss: 0.4034 - acc: 0.8350     
Epoch 18/70
1000/1000 [==============================] - 4s - loss: 0.3963 - acc: 0.8310     
Epoch 19/70
1000/1000 [==============================] - 4s - loss: 0.3716 - acc: 0.8460     
Epoch 20/70
1000/1000 [==============================] - 3s - loss: 0.3430 - acc: 0.8540     
Epoch 21/70
1000/1000 [==============================] - 3s - loss: 0.2944 - acc: 0.8880     
Epoch 22/70
1000/1000 [==============================] - 4s - loss: 0.2833 - acc: 0.8750     
Epoch 23/70
1000/1000 [==============================] - 4s - loss: 0.2426 - acc: 0.9070     
Epoch 24/70
1000/1000 [==============================] - 4s - loss: 0.2456 - acc: 0.8920     
Epoch 25/70
1000/1000 [==============================] - 4s - loss: 0.20

In [17]:
networkSR.save("networkSR.h5")
networkSD.save("siamNetSD.h5")
siamNetSD.save("siamNetSD.h5")

# networkSR = load_model('networkSR.h5')

In [15]:
perfSR=[]
perfSD=[]
predSD = networkSD.predict(imagesTe)
predSD=np.around(predSD)
resTeSD=np.subtract(SDlabelsTe.flatten(),predSD.flatten())
perfSD.append(1-sum(abs(resTeSD))/len(resTeSD))

predSR = networkSR.predict(imagesTe)
predSR=np.around(predSR)
resTeSR=np.subtract(SRlabelsTe.flatten(),predSR.flatten())
perfSR.append(1-sum(abs(resTeSR))/len(resTeSR))

print(np.mean(perfSD))
print(np.mean(perfSR))

0.5
0.999


In [16]:
perfSDsia=[]
predSDsia = siamNetSD.predict([imagesTeNewR,imagesTeNewL])
predSDsia=np.around(predSDsia)
resTeSDsia=np.subtract(SDlabelsTeSiam.flatten(),predSDsia.flatten())
perfSDsia.append(1-sum(abs(resTeSDsia))/len(resTeSDsia))
print(np.mean(perfSDsia))


0.954


In [11]:


sio.savemat('perfSD10sia.mat', mdict={'perfSDsia':perfSDsia})

historyAccSDsia=historySiam.history['acc']

sio.savemat('historyAccSDsia10.mat', mdict={'historyAccSDsia':historyAccSDsia})


In [46]:
networkSR.layers[18]

In [25]:
def grad_cam(x, model, sess, predicted_class, layer_name, nb_classes):
    print("Setting gradients to 1 for target class and rest to 0")
    # Conv layer tensor [?,7,7,512]
    conv_layer = model.layers[18]
    # [1000]-D tensor with target class index set to 1 and rest as 0
    one_hot = tf.sparse_to_dense(predicted_class, [nb_classes], 1.0)
    signal = tf.mul(vgg.layers['fc3'], one_hot)
    loss = tf.reduce_mean(signal)

    grads = tf.gradients(loss, conv_layer)[0]
    
# Normalizing the gradients
    norm_grads = tf.div(grads, tf.sqrt(tf.reduce_mean(tf.square(grads))) + tf.constant(1e-5))

    output, grads_val = sess.run([conv_layer, norm_grads], feed_dict={vgg.imgs: x})
    output = output[0]           # [7,7,512]
    grads_val = grads_val[0]	 # [7,7,512]

    weights = np.mean(grads_val, axis = (0, 1)) 			# [512]
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)	# [7,7]

    # Taking a weighted average
    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

# Passing through ReLU
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = resize(cam, (224,224))

    # Converting grayscale to 3-D
    cam3 = np.expand_dims(cam, axis=2)
    cam3 = np.tile(cam3,[1,1,3])

    return cam3

ModuleNotFoundError: No module named 'utils'

In [29]:
import tensorflow as tf
import numpy as np
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt
from scipy.misc import imread, imresize

In [ ]:
def grad_cam(x, vgg, sess, predicted_class, layer_name, nb_classes):
    print("Setting gradients to 1 for target class and rest to 0")
    
    # Conv layer tensor [?,7,7,512]
    conv_layer = vgg.layers[layer_name]
    
    # [1000]-D tensor with target class index set to 1 and rest as 0
    one_hot = tf.sparse_to_dense(predicted_class, [nb_classes], 1.0)
    signal = tf.mul(vgg.layers['fc3'], one_hot)
    loss = tf.reduce_mean(signal)

    grads = tf.gradients(loss, conv_layer)[0]
    
# Normalizing the gradients
    norm_grads = tf.div(grads, tf.sqrt(tf.reduce_mean(tf.square(grads))) + tf.constant(1e-5))

    output, grads_val = sess.run([conv_layer, norm_grads], feed_dict={vgg.imgs: x})
    output = output[0]           # [7,7,512]
    grads_val = grads_val[0]	 # [7,7,512]

    weights = np.mean(grads_val, axis = (0, 1)) 			# [512]
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)	# [7,7]

    # Taking a weighted average
    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

# Passing through ReLU
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = resize(cam, (224,224))

    # Converting grayscale to 3-D
    cam3 = np.expand_dims(cam, axis=2)
    cam3 = np.tile(cam3,[1,1,3])

    return cam3